In [4]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
from nltk.corpus import stopwords
# nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
import tensorflow as tf
from tensorflow import keras
import os
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot


In [7]:
df = pd.read_csv("./server/datasets/train.csv")

split = 0.8
df = df.sample(frac=1).dropna()
train = df[:int(0.8*df.shape[0])]
test = df[int(0.8*df.shape[0]):]

x_train, y_train = train["title"], train["label"]
x_test, y_test = test["title"], test["label"]

x_train_enc = [one_hot(words,10000) for words in x_train]
x_test_enc = [one_hot(words,10000) for words in x_test]

sent_length=10
x_train_enc_padded = pad_sequences(x_train_enc,padding='pre',maxlen=sent_length)

x_test_enc_padded = pad_sequences(x_test_enc,padding='pre',maxlen=sent_length)

In [ ]:
x_train_enc_padded.shape

(14628, 10)

In [8]:
def create_model():
    model = tf.keras.Sequential([
    keras.layers.Dense(512, activation = "relu", input_shape=(10, )),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
    ])
    model.compile(optimizer = "adam", loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
    return model

In [9]:
model = create_model()

In [10]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               5632      
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 10762 (42.04 KB)
Trainable params: 10762 (42.04 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
checkpoint_path = "training_1/checkpoints"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose=1)


In [14]:
model.fit(x_train_enc_padded, y_train, epochs = 20, validation_data=(x_test_enc_padded, y_test), callbacks=[cp_callback])


Epoch 1/20
458/458 [==============================] - ETA: 0s - loss: 0.5654 - sparse_categorical_accuracy: 0.7391
Epoch 1: saving model to training_1/checkpoints
458/458 [==============================] - 1s 1ms/step - loss: 0.5654 - sparse_categorical_accuracy: 0.7391 - val_loss: 0.5668 - val_sparse_categorical_accuracy: 0.7066
Epoch 2/20
430/458 [===========================>..] - ETA: 0s - loss: 0.6006 - sparse_categorical_accuracy: 0.7261
Epoch 2: saving model to training_1/checkpoints
458/458 [==============================] - 1s 1ms/step - loss: 0.5990 - sparse_categorical_accuracy: 0.7250 - val_loss: 0.5320 - val_sparse_categorical_accuracy: 0.7583
Epoch 3/20
403/458 [=========================>....] - ETA: 0s - loss: 0.5726 - sparse_categorical_accuracy: 0.7392
Epoch 3: saving model to training_1/checkpoints
458/458 [==============================] - 0s 1ms/step - loss: 0.5713 - sparse_categorical_accuracy: 0.7421 - val_loss: 0.5498 - val_sparse_categorical_accuracy: 0.7654
Epoc

In [15]:
loss, acc = model.evaluate(x_test_enc_padded,y_test, verbose=2)
print(f"model, accuracy: {100*acc:5.2f}%")

115/115 - 0s - loss: 0.5266 - sparse_categorical_accuracy: 0.7547 - 80ms/epoch - 696us/step
model, accuracy: 75.47%


In [16]:
os.listdir(checkpoint_dir)

['checkpoints.data-00000-of-00001', 'checkpoint', 'checkpoints.index']

In [18]:
model_loaded = create_model()

In [19]:
loss_loaded, acc_loaded = model_loaded.evaluate(x_test_enc_padded, y_test)
print(f"Untrained model, accuracy: {100*acc_loaded:5.2f}%")


115/115 [==============================] - 0s 652us/step - loss: 617.9942 - sparse_categorical_accuracy: 0.5228
Untrained model, accuracy: 52.28%


In [21]:
model_loaded.load_weights(checkpoint_path)

In [22]:
loss_loaded_trained, acc_loaded_trained = model_loaded.evaluate(x_test_enc_padded, y_test)
print(f"Trained model, accuracy: {100*acc_loaded_trained:5.2f}%")

115/115 [==============================] - 0s 559us/step - loss: 0.5266 - sparse_categorical_accuracy: 0.7547
Trained model, accuracy: 75.47%
